In [1]:
import time
import base64
from datetime import datetime, timedelta
import json
import logging
import requests
import streamlit as st
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
from cryptography.fernet import Fernet
import json
from typing import List, Any, Dict
from tqdm import tqdm

## Setup

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)

In [3]:
def send_bigquery(df: pd.DataFrame, table_id: str, schema: list):
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE",
    )

    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    table = client.get_table(table_id)
    print(
        "Loaded {} rows and {} columns to {} in {}".format(
            table.num_rows, len(table.schema), table_id, datetime.now()
        )
    )

    print("Send to biquery !")

In [4]:
def query_bigquery(sql_query: str):
    query_job = client.query(sql_query)

    return query_job.result()

In [5]:
def decrypt_password(encrypted_password, key):
    fernet = Fernet(key)
    decrypted_password = fernet.decrypt(encrypted_password).decode()
    return decrypted_password

In [6]:
def encrypt_password(password, key):
    fernet = Fernet(key)
    encrypted_password = fernet.encrypt(password.encode())
    return encrypted_password

In [7]:
class api_bling:
    def __init__(self):
        self.cache = {}
        self._401_count = 0

    def get(self, url, loja: str):
        try:
            titulo_loja = "".join(loja.split()).upper()
            acess_token = self._access_token(titulo_loja)

            headers = {
                "Accept": "application/json",
                "Authorization": f"Bearer {acess_token}",
            }

            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 401:
                if self._401_count >= 2:
                    response.raise_for_status()

                self._401_count += 1
                self.cache.clear()
                return self.get(url, loja)
            elif response.status_code == 429:
                time.sleep(1)
                return self.get(url, loja)
            else:
                response.raise_for_status()
        except Exception as e:
            print("Ocorreu um erro:", e)
            return "error"

    def post(self, url, body, loja: str):
        try:
            titulo_loja = "".join(loja.split()).upper()
            acess_token = self._access_token(titulo_loja)

            headers = {
                "Content-Type": "application/json",
                "Accept": "application/json",
                "Authorization": f"Bearer {acess_token}",
            }
            payload = json.dumps(body)
            response = requests.post(url, headers=headers, data=payload)

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 401:
                if self._401_count >= 2:
                    response.raise_for_status()

                self._401_count += 1
                self.cache.clear()
                return self.post(url, loja)
            elif response.status_code == 429:
                time.sleep(1)
                return self.post(url, loja)
            else:
                response.raise_for_status()
        except Exception as e:
            print("Ocorreu um erro:", e)
            return "error"

    def _oauth_refresh(self, df_credenciais: pd.DataFrame, loja: str) -> str:
        chave_criptografia = str(st.secrets["chave_criptografia"]).encode()
        bling_client_id = st.secrets[f"BLING_CLIENT_ID_{loja}"]
        bling_client_secret = st.secrets[f"BLING_CLIENT_SECRET_{loja}"]

        refresh_token = decrypt_password(
            str(
                df_credenciais["valor"]
                .loc[
                    (df_credenciais["titulo"] == "refresh_token")
                    & (df_credenciais["loja"] == f"BLING_{loja}")
                ]
                .values[0]
            ),
            chave_criptografia,
        )

        credentialbs4 = f"{bling_client_id}:{bling_client_secret}"
        credentialbs4 = credentialbs4.encode("ascii")
        credentialbs4 = base64.b64encode(credentialbs4)
        credentialbs4 = bytes.decode(credentialbs4)

        header = {"Accept": "1.0", "Authorization": f"Basic {credentialbs4}"}
        dice = {"grant_type": "refresh_token", "refresh_token": refresh_token}

        api = requests.post(
            "https://www.bling.com.br/Api/v3/oauth/token", headers=header, json=dice
        )

        situationStatusCode = api.status_code
        api = api.json()

        if situationStatusCode == 400:
            print(f"Request failed. code: {situationStatusCode}")

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "access_token"),
            "valor",
        ] = encrypt_password(api["access_token"], chave_criptografia)

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "access_token"),
            "validade",
        ] = str(datetime.now())

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "refresh_token"),
            "valor",
        ] = encrypt_password(api["refresh_token"], chave_criptografia)

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "refresh_token"),
            "validade",
        ] = str(datetime.now())

        schema = [
            bigquery.SchemaField("loja", "STRING"),
            bigquery.SchemaField("titulo", "STRING"),
            bigquery.SchemaField("validade", "STRING"),
            bigquery.SchemaField("valor", "STRING"),
        ]
        table_id = f"integracao-414415.data_ptl.credenciais"

        send_bigquery(df_credenciais, table_id, schema)

        return api["access_token"]

    def _validade_access_token(self, df_credenciais: pd.DataFrame, loja: str) -> str:

        data_atualizacao = datetime.strptime(
            df_credenciais["validade"]
            .loc[
                (df_credenciais["titulo"] == "access_token")
                & (df_credenciais["loja"] == f"BLING_{loja}")
            ]
            .values[0],
            "%Y-%m-%d %H:%M:%S.%f",
        )

        data_limite = data_atualizacao + timedelta(hours=6)

        if datetime.now() > data_limite or self._401_count >= 2:
            return self._oauth_refresh(df_credenciais, loja)

        return decrypt_password(
            df_credenciais["valor"]
            .loc[
                (df_credenciais["titulo"] == "access_token")
                & (df_credenciais["loja"] == f"BLING_{loja}")
            ]
            .values[0],
            str(st.secrets["chave_criptografia"]).encode(),
        )

    def _access_token(self, loja: str) -> str:
        if loja in self.cache:
            return self.cache[loja]

        results_query_credenciais = query_bigquery(
            "SELECT * FROM `integracao-414415.data_ptl.credenciais`"
        )

        df_credenciais = pd.DataFrame(
            data=[row.values() for row in results_query_credenciais],
            columns=[field.name for field in results_query_credenciais.schema],
        )

        self.cache[loja] = self._validade_access_token(df_credenciais, loja)

        return self.cache[loja]


In [8]:
produtos_bling = query_bigquery(
    "SELECT id, sku, nome, estoque_maximo, loja, variacao, estrutura FROM `integracao-414415.data_ptl.produtos_bling_proteloja`"
)

df_produtos_bling = pd.DataFrame(
    data=[row.values() for row in produtos_bling],
    columns=[field.name for field in produtos_bling.schema],
)

saldo_estoque_bling = query_bigquery(
    "SELECT id, saldo_fisico_total, estoque_matriz, full_amazon, full_mgl_proteloja, full_ml_proteloja, full_ml_vendolandia, full_ml_vendolandia2 FROM `integracao-414415.data_ptl.saldo_estoque_bling`"
)

df_saldo_estoque_bling = pd.DataFrame(
    data=[row.values() for row in saldo_estoque_bling],
    columns=[field.name for field in saldo_estoque_bling.schema],
)

fornecedores_bling = query_bigquery(
    "SELECT id_produto, id_fornecedor, nome_fornecedor, custo FROM `integracao-414415.data_ptl.fornecedores_bling_proteloja` WHERE padrao != false"
)

df_fornecedores_bling = pd.DataFrame(
    data=[row.values() for row in fornecedores_bling],
    columns=[field.name for field in fornecedores_bling.schema],
)

dia_entrega_fornecedor = query_bigquery(
    "SELECT dias_entrega_fornecedor, dias_entrega_full, id FROM `integracao-414415.data_ptl.dia_entrega_fornecedor`"
)

df_dia_entrega_fornecedor = pd.DataFrame(
    data=[row.values() for row in dia_entrega_fornecedor],
    columns=[field.name for field in dia_entrega_fornecedor.schema],
)


## Saldo Estoque

In [9]:
def get_saldo_estoque(loja: str, lista_produtos: List[str]) -> List[Dict[str, Any]]:
    api = api_bling()
    dados_estoque = []

    print(len(lista_produtos))

    for i in tqdm(range(0, len(lista_produtos), 5)):
        time.sleep(0.1)

        selecao = "&idsProdutos[]="+"&idsProdutos[]=".join(lista_produtos[i : i + 5])
        # selecao = f"&idsProdutos[]={lista_produtos[i]}"

        try:
            response = api.get(
                f"https://www.bling.com.br/Api/v3/estoques/saldos?{selecao}",
                loja,
            )

            if response == "error" or response.get("data", []) == []:
                print(response.txt)

            for produto in response.get("data", []):
                id_produto: str = str(produto.get("produto", {}).get("id", ""))
                saldo_fisico_total: float = float(produto.get("saldoFisicoTotal", 0))

                saldo_estoques = {
                    2816599510: 0,
                    11910799658: 0,
                    14887642988: 0,
                    9738790725: 0,
                    14197230585: 0,
                    14886665514: 0,
                }

                for deposito in produto.get("depositos", []):
                    saldo_estoques[deposito.get("id", 0)] = float(
                        deposito.get("saldoFisico", 0)
                    )

                dados_estoque.append(
                    {
                        "id": id_produto,
                        "saldo_fisico_total": saldo_fisico_total,
                        "estoque_matriz": saldo_estoques[2816599510],
                        "full_amazon": saldo_estoques[11910799658],
                        "full_mgl_proteloja": saldo_estoques[14887642988],
                        "full_ml_proteloja": saldo_estoques[9738790725],
                        "full_ml_vendolandia": saldo_estoques[14197230585],
                        "full_ml_vendolandia2": saldo_estoques[14886665514],
                    }
                )

        except Exception as e:
            print("Error get_fornecedores_bling /fornecedores: ", e)

    return dados_estoque

In [10]:
def atualizar_saldo_estoque(
    loja: str, lista_id_produtos: List[str]
) -> List[Dict[str, Any]]:
    df_saldo_estoque_bling = pd.DataFrame(get_saldo_estoque(loja, lista_id_produtos))

    if not df_saldo_estoque_bling.empty:
        df_saldo_estoque_bling.fillna(0, inplace=True)

        schema = [
            bigquery.SchemaField("id", "STRING"),
            bigquery.SchemaField("saldo_fisico_total", "FLOAT"),
            bigquery.SchemaField("estoque_matriz", "FLOAT"),
            bigquery.SchemaField("full_amazon", "FLOAT"),
            bigquery.SchemaField("full_mgl_proteloja", "FLOAT"),
            bigquery.SchemaField("full_ml_proteloja", "FLOAT"),
            bigquery.SchemaField("full_ml_vendolandia", "FLOAT"),
            bigquery.SchemaField("full_ml_vendolandia2", "FLOAT"),
        ]
        table_id = f"integracao-414415.data_ptl.saldo_estoque_bling"

        send_bigquery(df_saldo_estoque_bling, table_id, schema)

## Pedidos de compra

In [56]:
from typing import List, Any, Dict

def get_pedidos_compra_bling(loja: str) -> List[Dict[str, Any]]:
    api = api_bling()
    id_pedidos = []
    dados_pedidos = []
    page = 1

    today = datetime.now().date()
    ontem = today - timedelta(days=90)

    data_anterior = ontem.strftime("%Y-%m-%d")
    data_posterior = today.strftime("%Y-%m-%d")

    while True:
        try:
            response = api.get(
                f"https://www.bling.com.br/Api/v3/pedidos/compras?pagina={page}&limite=100&idSituacao=28&dataInicial={data_anterior}&dataFinal={data_posterior}",
                loja,
            )

            if response == "error" or response["data"] == []:
                break

            for i in response["data"]:
                id_pedidos.append(i.get("id", ""))

            page += 1
        except Exception as e:
            print("Error get_pedidos_compra_bling /compras: ", e)

    if id_pedidos == []:
        return []

    for pedido in id_pedidos:
        if pedido != "" and pedido != None:
            try:
                response = api.get(
                    f"https://www.bling.com.br/Api/v3/pedidos/compras/{pedido}", loja
                )

                if response == []:
                    continue

                pedido: list = response.get("data", [])

                for item in pedido.get("itens", {}):
                    id_pedido: str = str(pedido.get("id", ""))
                    numero_pedido: str = str(pedido.get("numero", ""))
                    data: str = str(pedido.get("data", ""))
                    data_prevista: str = str(pedido.get("dataPrevista", ""))
                    total: float = float(pedido.get("total", 0))
                    fornecedor: str = str(pedido.get("fornecedor", {}).get("id", ""))
                    situacao: str = str(pedido.get("situacao", {}).get("valor", ""))
                    nome_produto: str = str(item.get("descricao", ""))
                    valor_produto: float = float(item.get("valor", 0))
                    quantidade: float = float(item.get("quantidade", 0))
                    id_produto: str = str(item.get("produto", {}).get("id", ""))
                    sku: str = str(item.get("produto", {}).get("codigo", ""))

                    dados_pedidos.append(
                        {
                            "id_pedido": id_pedido,
                            "numero_pedido": numero_pedido,
                            "data": data,
                            "data_prevista": data_prevista,
                            "total": total,
                            "fornecedor": fornecedor,
                            "situacao": situacao,
                            "nome_produto": nome_produto,
                            "valor_produto": valor_produto,
                            "quantidade": quantidade,
                            "id_produto": id_produto,
                            "sku": sku,
                        }
                    )

            except Exception as e:
                print("Error get_pedidos_compra_bling /compras/pedido: ", e)

    return dados_pedidos

### Analise compras

In [ ]:
df_compras = pd.DataFrame(get_pedidos_compra_bling("PROTELOJA"))

In [26]:
df_compras.head()

,id_pedido,numero_pedido,data,data_prevista,total,fornecedor,situacao,nome_produto,valor_produto,quantidade,id_produto,sku
0,20490024158,2813,2024-07-22,2024-10-25,9300.00,9601723168,0,PERNEIRA BIDIN COM VELCRO 3 TALAS Tamanho:Único,15.50,600.0,4267438194,6586-V
1,20717927134,2888,2024-07-16,2024-09-18,1997.50,4744461407,0,POLIA ROLDANA ALUMINIO 20KN LIFE-5300 Tamanho:...,79.90,25.0,15562987373,10298-V
2,20760098140,2903,2024-07-23,2024-09-16,1713.60,4279308291,0,LUVA ALTA TEMPERATURA EM COURO VERMELHO CA615K...,28.56,60.0,14636992584,9863-V
3,20847038396,2929,2024-08-05,2024-10-04,3291.84,16136735406,0,TESOURA USO GERAL TRAMONTINA PROFISSIONAL Tama...,11.43,288.0,9172554354,8704-V
4,20848882704,2932,2024-08-05,2024-09-20,16740.00,4846660898,0,MÁSCARA PFF2 COM VÁLVULA N95 Tamanho:Único,0.62,27000.0,10040119562,9309-V


In [29]:
df_compras["situacao"].unique()

array(['0'], dtype=object)

In [32]:
df_compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_pedido      56 non-null     object 
 1   numero_pedido  56 non-null     object 
 2   data           56 non-null     object 
 3   data_prevista  56 non-null     object 
 4   total          56 non-null     float64
 5   fornecedor     56 non-null     object 
 6   situacao       56 non-null     object 
 7   nome_produto   56 non-null     object 
 8   valor_produto  56 non-null     float64
 9   quantidade     56 non-null     float64
 10  id_produto     56 non-null     object 
 11  sku            56 non-null     object 
dtypes: float64(3), object(9)
memory usage: 5.4+ KB


In [33]:
df_compras[df_compras["sku"] == "9309-V"]

,id_pedido,numero_pedido,data,data_prevista,total,fornecedor,situacao,nome_produto,valor_produto,quantidade,id_produto,sku
4,20848882704,2932,2024-08-05,2024-09-20,16740.0,4846660898,0,MÁSCARA PFF2 COM VÁLVULA N95 Tamanho:Único,0.62,27000.0,10040119562,9309-V
22,21020829238,2976,2024-08-30,2024-10-04,34100.0,4846660898,0,MÁSCARA PFF2 COM VÁLVULA N95 Tamanho:Único,0.62,55000.0,10040119562,9309-V
53,21113760302,2999,2024-09-13,2024-11-08,18600.0,4846660898,0,MÁSCARA PFF2 COM VÁLVULA N95 Tamanho:Único,0.62,30000.0,10040119562,9309-V


In [ ]:
df_compras['data_prevista'] = pd.to_datetime(df_compras['data_prevista'])
df_min_dates = df_compras.groupby('id_produto')['data_prevista'].min().reset_index()
df_min = pd.merge(df_min_dates, df_compras, on=['id_produto', 'data_prevista'], how='outer')
df_resultado_compras = df_min[['id_produto', 'quantidade', 'data_prevista']]

df = pd.merge(df, df_resultado_compras, on="id_produto", how='outer')

## Relatorio

In [11]:
# transformando dataframes requisitados por sql
df_produtos_bling["id"] = df_produtos_bling["id"].astype(str)
df_saldo_estoque_bling["id"] = df_saldo_estoque_bling["id"].astype(str)

df_produtos_bling = df_produtos_bling[df_produtos_bling["variacao"] != "{}"]
df_produtos_bling = df_produtos_bling[df_produtos_bling["estrutura"] == "{'tipoEstoque': '', 'lancamentoEstoque': '', 'componentes': []}"]
df_produtos_bling = df_produtos_bling[
    ~df_produtos_bling["nome"].str.contains("CONSUMIVEL", case=False, na=False)
]
df_produtos_bling = df_produtos_bling[df_produtos_bling["loja"] == "proteloja"]

In [15]:
df_produtos_bling["id"].array

<NumpyExtensionArray>
[ '8650657941',  '8650657934',  '8650657907',  '8650657947',  '8650694468',
 '15899533363',  '4267438449', '15858733674', '16173438421', '15858733676',
 ...
 '15937522588', '15676716383', '14208034935',  '4267434477',  '9592273181',
  '9592273178',  '9592273188',  '4267438194', '16095374877', '16022344649']
Length: 173, dtype: object

In [183]:
# requisitando e transformando pedidos de compra
df_compras = pd.DataFrame(get_pedidos_compra_bling("PROTELOJA"))
df_compras['data_prevista'] = pd.to_datetime(df_compras['data_prevista'])
df_min_dates = df_compras.groupby('id_produto')['data_prevista'].min().reset_index()
df_min = pd.merge(df_min_dates, df_compras, on=['id_produto', 'data_prevista'])
df_resultado_compras = df_min[['id_produto', 'quantidade', 'data_prevista']]

In [184]:
# juntando e ajustando dataframes
df = pd.merge(df_produtos_bling, df_saldo_estoque_bling, on="id", how='left')
df = pd.merge(df, df_dia_entrega_fornecedor, on="id", how='left')
df = df.rename(
    columns={
        "id": "id_produto",
        "estoque_maximo": "50 dias",
        "saldo_fisico_total": "estoque total",
        "estoque_matriz": "matriz",
        "full_amazon": "full amz",
        "full_mgl_proteloja": "full magalu",
        "full_ml_proteloja": "full prot",
        "full_ml_vendolandia": "full vend",
        "full_ml_vendolandia2": "full vend2"
    }
)

df = pd.merge(df, df_fornecedores_bling, on="id_produto", how='left')
df = pd.merge(df, df_resultado_compras, on="id_produto", how='left')
df = df.rename(
    columns={
        "dias_entrega_fornecedor": "dias entrega fornecedor",
        "dias_entrega_full": "dias entrega full",
        "nome_fornecedor": "nome fornecedor",
        "quantidade": "quantidade comprada",
        "data_prevista": "entrega prevista"
    }
)

df.drop(["id_produto", "id_fornecedor", "loja", "variacao", "estrutura"], axis=1, inplace=True)

In [185]:
#convertendo coluna datas
df['entrega prevista'] = df['entrega prevista'].dt.strftime('%Y/%m/%d')

# calculando dias de estoque 
df["dias de estoque"] = round(df["estoque total"] / (df["50 dias"] / 50))

#adicionando status do estoque
df.insert(0, "checkbox", [False] * len(df))

#removendo valores nulos
df.fillna(0, inplace=True)

In [186]:
#adicionando status
df['status'] = [[] for _ in range(len(df))]

for index, row in df.iterrows():
    dias_compras =  (pd.to_datetime(row['entrega prevista']).date() - datetime.today().date()).days if pd.to_datetime(row['entrega prevista']).date() > datetime.today().date() else 7
    
    tempo_entrega = row["dias entrega fornecedor"] + row["dias entrega full"] 

    media_vendas_diaria = row["50 dias"] / 50

    estoque_seguranca = media_vendas_diaria * 25
    estoque_maximo = estoque_seguranca + (media_vendas_diaria * tempo_entrega)

    estoque_futuro = lambda dias: (row["estoque total"] - (media_vendas_diaria * dias)) 
    
    if row['entrega prevista'] != 0:
        if (estoque_futuro(dias_compras) + row["quantidade comprada"]) > estoque_maximo:
            df.at[index, 'status'].append('comprou muito')
    
        if estoque_futuro(dias_compras) < estoque_seguranca:
            df.at[index, 'status'].append('ruptura')

    else:
        if estoque_futuro(tempo_entrega) < estoque_seguranca:
            df.at[index, 'status'].append('ruptura')

    if row['estoque total'] > estoque_maximo:
        df.at[index, 'status'].append('estoque sobrecarregado')

    if row['dias de estoque'] <= 7:
        df.at[index, 'status'].append('verificar compras')


In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   checkbox                 174 non-null    bool   
 1   sku                      174 non-null    object 
 2   nome                     174 non-null    object 
 3   50 dias                  174 non-null    float64
 4   estoque total            174 non-null    float64
 5   matriz                   174 non-null    float64
 6   full amz                 174 non-null    float64
 7   full magalu              174 non-null    float64
 8   full prot                174 non-null    float64
 9   full vend                174 non-null    float64
 10  full vend2               174 non-null    float64
 11  dias entrega fornecedor  174 non-null    float64
 12  dias entrega full        174 non-null    float64
 13  nome fornecedor          174 non-null    object 
 14  custo                    1

In [187]:
df.head()

,checkbox,sku,nome,50 dias,estoque total,matriz,full amz,full magalu,full prot,full vend,full vend2,dias entrega fornecedor,dias entrega full,nome fornecedor,custo,quantidade comprada,entrega prevista,dias de estoque,status
0,False,8283-G,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:G,998.0,347.0,38.0,0.0,-9.0,268.0,14.0,27.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,1100.0,2024/09/16,17.0,[ruptura]
1,False,8283-M,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:M,1368.0,669.0,62.0,0.0,450.0,568.0,18.0,21.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,1350.0,2024/09/16,24.0,[ruptura]
2,False,8283-P,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:P,511.0,273.0,31.0,0.0,7.0,204.0,3.0,35.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,300.0,2024/09/16,27.0,[ruptura]
3,False,8283-GG,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:GG,485.0,247.0,71.0,0.0,-4.0,161.0,5.0,10.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,350.0,2024/09/16,25.0,[ruptura]
4,False,8283-EXG,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:EXG,279.0,239.0,87.0,0.0,21.0,136.0,7.0,9.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,100.0,2024/09/16,43.0,[]


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   checkbox                 174 non-null    bool   
 1   sku                      174 non-null    object 
 2   nome                     174 non-null    object 
 3   50 dias                  174 non-null    float64
 4   estoque total            174 non-null    float64
 5   matriz                   174 non-null    float64
 6   full amz                 174 non-null    float64
 7   full magalu              174 non-null    float64
 8   full prot                174 non-null    float64
 9   full vend                174 non-null    float64
 10  full vend2               174 non-null    float64
 11  dias entrega fornecedor  174 non-null    float64
 12  dias entrega full        174 non-null    float64
 13  nome fornecedor          174 non-null    object 
 14  custo                    1

In [176]:
#adicionando status
df['status'] = [[] for _ in range(len(df))]

for index, row in df.iterrows():
    dias_compras =  (pd.to_datetime(row['entrega prevista'], format='%d/%m/%Y').date() - datetime.today().date()).days if pd.to_datetime(row['entrega prevista'], format='%d/%m/%Y').date() > datetime.today().date() else 7
    
    tempo_entrega = row["dias entrega fornecedor"] + row["dias entrega full"] 

    media_vendas_diaria = row["50 dias"] / 50

    estoque_seguranca = media_vendas_diaria * 25
    estoque_maximo = estoque_seguranca + (media_vendas_diaria * tempo_entrega)

    estoque_futuro = lambda dias: (row["estoque total"] - (media_vendas_diaria * dias)) 
    
    if row['entrega prevista'] != 0:
        if (estoque_futuro(dias_compras) + row["quantidade comprada"]) > estoque_maximo:
            df.at[index, 'status'].append('comprou muito')
    
        if estoque_futuro(dias_compras) < estoque_seguranca:
            df.at[index, 'status'].append('ruptura')

    else:
        if estoque_futuro(tempo_entrega) < estoque_seguranca:
            df.at[index, 'status'].append('ruptura')

    if row['estoque total'] > estoque_maximo:
        df.at[index, 'status'].append('estoque sobrecarregado')

    if row['dias de estoque'] <= 7:
        df.at[index, 'status'].append('verificar compras')

       

In [168]:
data = {
  'entrega prevista': "2024-09-16",
  "dias entrega fornecedor": 35.0,
  "dias entrega full": 8.0,
  "estoque total": 347.0,
  "50 dias": 998.0,
}

dias_compras_teste =  (datetime.strptime(data['entrega prevista'], "%Y-%m-%d").date() - datetime.today().date()).days if datetime.strptime(data['entrega prevista'], "%Y-%m-%d").date() > datetime.now().date() else 7

tempo_entrega_teste = data["dias entrega fornecedor"] + data["dias entrega full"] 

media_vendas_diaria_teste  = data["50 dias"] / 50

estoque_seguranca_teste  = media_vendas_diaria_teste  * 25
estoque_maximo_teste  = estoque_seguranca_teste  + (media_vendas_diaria_teste  * tempo_entrega_teste)

estoque_futuro_teste = lambda dias: (data["estoque total"] - (media_vendas_diaria_teste  * dias)) 

In [178]:
estoque_futuro_teste(dias_compras_teste)

207.28

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   checkbox                 174 non-null    bool   
 1   sku                      174 non-null    object 
 2   nome                     174 non-null    object 
 3   50 dias                  174 non-null    float64
 4   estoque total            174 non-null    float64
 5   matriz                   174 non-null    float64
 6   full amz                 174 non-null    float64
 7   full magalu              174 non-null    float64
 8   full prot                174 non-null    float64
 9   full vend                174 non-null    float64
 10  full vend2               174 non-null    float64
 11  dias entrega fornecedor  174 non-null    float64
 12  dias entrega full        174 non-null    float64
 13  nome fornecedor          174 non-null    object 
 14  custo                    1

In [177]:
df.head()

,checkbox,sku,nome,50 dias,estoque total,matriz,full amz,full magalu,full prot,full vend,full vend2,dias entrega fornecedor,dias entrega full,nome fornecedor,custo,quantidade comprada,entrega prevista,dias de estoque,status
0,False,8283-G,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:G,998.0,347.0,38.0,0.0,-9.0,268.0,14.0,27.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,1100.0,2024-09-16 00:00:00,17.0,[ruptura]
1,False,8283-M,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:M,1368.0,669.0,62.0,0.0,450.0,568.0,18.0,21.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,1350.0,2024-09-16 00:00:00,24.0,[ruptura]
2,False,8283-P,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:P,511.0,273.0,31.0,0.0,7.0,204.0,3.0,35.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,300.0,2024-09-16 00:00:00,27.0,[ruptura]
3,False,8283-GG,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:GG,485.0,247.0,71.0,0.0,-4.0,161.0,5.0,10.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,350.0,2024-09-16 00:00:00,25.0,[ruptura]
4,False,8283-EXG,CINTA ERGONOMICA ABDOMINAL PROTELOJA Tamanho:EXG,279.0,239.0,87.0,0.0,21.0,136.0,7.0,9.0,35.0,8.0,LIFECINTOS EQUIPAMENTOS INDUSTRIAIS LTDA,22.0,100.0,2024-09-16 00:00:00,43.0,[]


coluna status

comprou muito -> o que comprou + estoque quando chegar < estoque maximo
quanidade comprada de determinado produto

ruptura -> o estoque vai dicar menor do que o minimo quando a compra chegar, caso não tenha compra, calcular o dia de estoque de entrega

verificar compras -> quando o comprar estiver dentro de 7 dias

estoque sobrecarregado -> quando o estoque máximo estiver menor do que o estoque atual